In [1]:
import pandas as pd
import sqlite3
import json

# ==========================================
# STEP 0: LOAD RAW DATA
# ==========================================

In [5]:
# 1. Load CSV (Transactional Data)
orders_df = pd.read_csv("C:/Users/prasad/Downloads/orders.csv")

# 2. Load JSON (User Master Data)
users_df = pd.read_json("C:/Users/prasad/Downloads/users.json")

# 3. Load SQL (Restaurant Master Data)
# Since the data is in .sql format, we use an in-memory SQLite database to extract it
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
with open("C:/Users/prasad/Downloads/restaurants.sql", 'r') as f:
    sql_script = f.read()
    cursor.executescript(sql_script)

restaurants_df = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()

# ==========================================
# STEP 1: MERGE DATASETS (LEFT JOIN)
# ==========================================

In [6]:

# Join orders with users on 'user_id'
merged_data = orders_df.merge(users_df, on='user_id', how='left')

# Join with restaurants on 'restaurant_id'
# suffixes are added to handle overlapping column names (like 'restaurant_name')
df = merged_data.merge(restaurants_df, on='restaurant_id', how='left', suffixes=('', '_rest'))

# Export the final dataset for submission
df.to_csv('final_food_delivery_dataset.csv', index=False)

# ==========================================
# STEP 2: DATA PREPROCESSING
# ==========================================

In [7]:
# Convert 'order_date' to datetime objects
df['order_date'] = pd.to_datetime(df['order_date'], format='%d-%m-%Y')

# Extract Quarter for seasonal analysis
df['quarter'] = df['order_date'].dt.quarter

# ==========================================
# STEP 3: ANALYZING MULTIPLE CHOICE QUESTIONS
# ==========================================

In [8]:
print("--- MCQ ANALYSIS ---")

# Q1: City with highest total revenue from Gold members
gold_city_rev = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum()
print(f"Q1: Highest Gold Revenue City: {gold_city_rev.idxmax()}")

# Q2: Cuisine with highest Average Order Value (AOV)
cuisine_aov = df.groupby('cuisine')['total_amount'].mean()
print(f"Q2: Highest Cuisine AOV: {cuisine_aov.idxmax()}")

# Q3: Distinct users with total spend > ₹1000
user_spend = df.groupby('user_id')['total_amount'].sum()
high_spend_users = (user_spend > 1000).sum()
print(f"Q3: Users spending > 1000: {high_spend_users}")

# Q4: Rating range generating highest revenue
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0']
df['rating_range'] = pd.cut(df['rating'], bins=bins, labels=labels, include_lowest=True)
rating_rev = df.groupby('rating_range')['total_amount'].sum()
print(f"Q4: Highest Revenue Rating Range: {rating_rev.idxmax()}")

# Q5: Gold members' highest AOV city
gold_city_aov = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].mean()
print(f"Q5: Highest Gold AOV City: {gold_city_aov.idxmax()}")

# Q7: Percentage of orders by Gold members
gold_pct = (len(df[df['membership'] == 'Gold']) / len(df)) * 100
print(f"Q7: Gold Order Percentage: {round(gold_pct)}%")

# Q10: Highest revenue quarter
quarter_rev = df.groupby('quarter')['total_amount'].sum()
print(f"Q10: Highest Revenue Quarter: Q{quarter_rev.idxmax()}")

# ==========================================
# STEP 4: NUMERICAL ANSWERS
# ==========================================

print("\n--- NUMERICAL ANSWERS ---")

# 1. Total Gold orders
print(f"1. Total Gold orders: {len(df[df['membership'] == 'Gold'])}")

# 2. Hyderabad Revenue (rounded)
hyd_rev = df[df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"2. Hyderabad Revenue: {round(hyd_rev)}")

# 3. Distinct users
print(f"3. Distinct users count: {df['user_id'].nunique()}")

# 4. Gold AOV (2 decimals)
print(f"4. Gold Avg Order Value: {round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)}")

# 5. Orders for restaurants with rating >= 4.5
high_rating_count = len(df[df['rating'] >= 4.5])
print(f"5. Rating >= 4.5 orders: {high_rating_count}")

# 6. Orders in top Gold revenue city (Chennai)
top_gold_city_name = gold_city_rev.idxmax()
q6_orders = len(df[(df['city'] == top_gold_city_name) & (df['membership'] == 'Gold')])
print(f"6. Orders in {top_gold_city_name} (Gold): {q6_orders}")

--- MCQ ANALYSIS ---
Q1: Highest Gold Revenue City: Chennai
Q2: Highest Cuisine AOV: Mexican
Q3: Users spending > 1000: 2544
Q4: Highest Revenue Rating Range: 4.6-5.0
Q5: Highest Gold AOV City: Chennai
Q7: Gold Order Percentage: 50%
Q10: Highest Revenue Quarter: Q3

--- NUMERICAL ANSWERS ---
1. Total Gold orders: 4987
2. Hyderabad Revenue: 1889367
3. Distinct users count: 2883
4. Gold Avg Order Value: 797.15
5. Rating >= 4.5 orders: 3374
6. Orders in Chennai (Gold): 1337


C:\Users\prasad\AppData\Local\Temp\ipykernel_40200\4088012783.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_rev = df.groupby('rating_range')['total_amount'].sum()
